# Skywalker algorithm against the lcls mirror systems

In [1]:
SIMULATION = True
ALIGNMENT = 'HOMS'
#ALIGNMENT = 'MEC'
#ALIGNMENT = 'MFX'
#ALIGNMENT = 'XCS'

alignment_goals = dict(HOMS=[210, 270],
                       MEC=[270],
                       MFX=[270],
                       XCS=[270, 270])
first_steps = 10 # Naive first step for the parameters search
tolerances = 2.0 # Tolerance of each target in Pixels
average = 1 # Number of shots to average over
log_level = "INFO"

# PARAMETERS FOR SIMULATION ONLY
centroid_noise = 5.0 # Noise level of centroid measurements
infinite_yag = False # Assume all yags are infinitely large

In [2]:
############
# Standard #
############
import os
import sys
import logging
import argparse

###############
# Third Party #
###############
from bluesky import RunEngine
from bluesky.tests.utils import MsgCollector
from bluesky.plans import run_wrapper
from pcdsdevices.sim import pim, source, mirror

In [3]:
"""
Parameters
----------
"""
#Configure logger
log_level = getattr(logging, log_level, None)

#Report invalid logging level
if not isinstance(log_level, int):
    raise ValueError("Invalid log level : {}".format(log_level))

#Create basic configuration
logging.basicConfig(level=log_level, format='%(name)s - %(message)s')

from pswalker.examples  import patch_pims
from pswalker.skywalker import skywalker
from pswalker.plans import walk_to_pixel
from pswalker.watcher import Watcher
#Instantiate simulation
if not SIMULATION:
    print("*"*80)
    print("* WARNING: Running on real devices in Production...")
    print("*"*80)
    input("Press any key to continue...")
    system = homs_system()
    m1h = system['m1h']
    m1h2 = system['m1h2']
    m2h = system['m2h']
    m2h2 = system['m2h2']
    xrtm2 = system['xrtm2']
    xrtm22 = system['xrtm22']
    hx2 = system['hx2']
    dg3 = system['dg3']
    mfxdg1 = system['mfxdg1']
    mecy1 = system['mecy1']
    m1 = m1h
    m2 = m2h
    y1 = hx2
    y2 = dg3
else:
    s = source.Undulator('test_undulator')
    m1 = mirror.OffsetMirror('test_m1h', 'test_m1h_xy',
                              z=90.510, alpha=0.0014)
    m2 = mirror.OffsetMirror('test_m2h', 'test_m2h_xy',
                              x=0.0317324, z=101.843, alpha=0.0014)
    xrtm2 = mirror.OffsetMirror('test_xrtm2', 'test_xrtm2_xy',
                                 x=0.0317324, z=200, alpha=0.0014)
    y1 = pim.PIM('test_p3h', x=0.0317324, z=103.660,
             zero_outside_yag= not infinite_yag)
    y2 = pim.PIM('test_dg3', x=0.0317324, z=375.000,
             zero_outside_yag= not infinite_yag)
    mecy1 = pim.PIM('test_mecy1', x=0.0317324, z=350,
             zero_outside_yag= not infinite_yag)
    mfxdg1 = mecy1
    patch_pims([y1, y2], mirrors=[m1, m2], source=s)
    patch_pims([mecy1], mirrors=[xrtm2], source=s)

    #Add noise
    y1.centroid_noise = centroid_noise
    y2.centroid_noise = centroid_noise
    mecy1.centroid_noise = centroid_noise

alignment_yags = dict(HOMS=[y1, y2],
                      MEC=[mecy1],
                      MFX=[mfxdg1],
                      XCS=['pbty', 'xcssb1y'])
alignment_mots = dict(HOMS=[m1, m2],
                      MEC=[xrtm2],
                      MFX=[xrtm2],
                      XCS=['xrtm1', 'xrtm3'])

In [4]:
#Create Skywalker plan
yags = alignment_yags[ALIGNMENT]
mots = alignment_mots[ALIGNMENT]
goals = alignment_goals[ALIGNMENT]
if SIMULATION:
    det_rbv = 'detector_stats2_centroid_x'
else:
    det_rbv = 'detector_stats2_centroid_y'
mot_rbv = 'pitch'
plan = skywalker(yags, mots, det_rbv, mot_rbv,
        goals, first_steps=first_steps, tolerances=tolerances,
        averages=average, timeout=10)  

In [5]:
#Create RunEngine
RE = RunEngine({})
RE.record_interruptions = True
#Subscribe a Watcher instance
watcher = Watcher()
RE.msg_hook = watcher
RE.subscribe('all',  watcher)


0

In [6]:
RE(plan)

pswalker.iterwalk - Starting walk from 2992.092302008178 to 270 on TST:test_p3h using MIRR:TST:test_m1h
pswalker.plans - Initial error before fitwalk is -2720
pswalker.plans - No model yielded accurate prediction, using naive plan
pcdsdevices.epics.mirror - Waiting for MIRR:TST:test_m1h_pitch to finish move ...
pswalker.plans - fitwalk is reporting an error -2700 of after step #1
pswalker.plans - Using model Model(linear) to determine next step.
pswalker.plans - Adjusting motor MIRR:TST:test_m1h to position 1321.3
pcdsdevices.epics.mirror - Waiting for MIRR:TST:test_m1h_pitch to finish move ...
pswalker.plans - fitwalk is reporting an error 203 of after step #2
pswalker.plans - Using model Model(linear) to determine next step.
pswalker.plans - Adjusting motor MIRR:TST:test_m1h to position 1229.2
pcdsdevices.epics.mirror - Waiting for MIRR:TST:test_m1h_pitch to finish move ...
pswalker.plans - Succesfully walked to value 270 (target=270) after 3 steps.
pswalker.iterwalk - Starting walk 

['1ed26407-01db-4fc7-8374-1686b85ed098']

In [7]:
#Print Run Statistics
watcher.report()


Skywalker successfuly aligned MIRR:TST:test_m1h, MIRR:TST:test_m2h to 210, 270
on TST:test_p3h, TST:test_dg3 in 0.0 seconds! The entire alignment moved the
mirrors 14 times, cycling between YAGs 9 times. The user requested that the
mirrors hit their targets within 2.0 pixels, averaging over 1 consecutive
images after each mirror motion. The resulting locations of mirrors and
centroid measurements can be seen in the table below.

+-----------------------------------------+---------------------+
|                                   Field | Last Measured Value |
+-----------------------------------------+---------------------+
| TST:test_p3h_detector_stats2_centroid_x |           269.22292 |
|                 MIRR:TST:test_m1h_pitch |          1425.33470 |
| TST:test_dg3_detector_stats2_centroid_x |           208.22305 |
|                 MIRR:TST:test_m2h_pitch |          1423.97923 |
+-----------------------------------------+---------------------+


'\nSkywalker successfuly aligned MIRR:TST:test_m1h, MIRR:TST:test_m2h to 210, 270\non TST:test_p3h, TST:test_dg3 in 0.0 seconds! The entire alignment moved the\nmirrors 14 times, cycling between YAGs 9 times. The user requested that the\nmirrors hit their targets within 2.0 pixels, averaging over 1 consecutive\nimages after each mirror motion. The resulting locations of mirrors and\ncentroid measurements can be seen in the table below.\n\n+-----------------------------------------+---------------------+\n|                                   Field | Last Measured Value |\n+-----------------------------------------+---------------------+\n| TST:test_p3h_detector_stats2_centroid_x |           269.22292 |\n|                 MIRR:TST:test_m1h_pitch |          1425.33470 |\n| TST:test_dg3_detector_stats2_centroid_x |           208.22305 |\n|                 MIRR:TST:test_m2h_pitch |          1423.97923 |\n+-----------------------------------------+---------------------+'